# TP3 : Modèles de langage appliqués à la traduction automatique

<i>TP adapté de ceux de Yongxin Zhou, eux-même inspiré du TP de ALPS 2021, écrit par Alexandre Bérard</i>

**Date limite de rendu :** Le 13 octobre 2024 à 10:30.

Ce TP ne demande pas d'écriture de code. Il est en revanche plus long et contient beaucoup plus de questions que le précédent. Pour y répondre, inutile d'écrire beaucoup de texte : une ou deux phrases suffisent en général à chaque question.

De plus, il n'est pas nécessaire que vous lisiez en détail chaque ligne du code qui est donné, mais il faut que vous compreniez ce qu'il se passe dans les grandes lignes. Les questions posées sont justement prévues pour cela.

-----------------------

### But du TP
Dans ce TP, nous allons entraîner et utiliser des modèles de langage pour la traduction automatique. Contrairement aux précédents TP, il ne s'agira plus d'une tâche de classification simple (données d'entrée -> classe) ; cette fois, nous disposons d'une *séquence* d'entrée (la phrase source) que nous souhaitons transformer en une séquende de sortie (la phrase traduite). Le type d'architecture neuronale que nous allons utiliser pour cela s'appelle donc en toute logique *seq2seq* (séquence vers séquence).

## Préparatifs

**Important :** Ce TP requiert d'avoir un GPU afin d'accélerer l'entraînement, sinon celui-ci dure beaucoup trop longtemps. Il est donc fortement conseillé d'exécuter ce notebook sur Google Colab.
Avant de commencer à exécuter les cellules suivantes ou d'importer les fichiers, il est également nécessaire de changer les paramètres de Colab afin de s'assurer d'avoir accès à un GPU. Cela peut se faire à l'aide du menu `Session > Changer le type de session` et en sélectionnant `GPU T4`.

Comme au TP précédent, vous aurez besoin d'importer les fichiers auxilliaires (`nmt_dataset.py`, `nnet_models.py`, `prepare.py`) ainsi que les images dans l'instance Colab.

Les commandes suivantes permettent d'installer et de charger les bibliothèques Python nécessaires. Comme au cours du TP précédent, n'oubliez pas de redémarrer le noyau Python après installation.

In [ ]:
# Installation des librairies
!pip install torch jupyter subword-nmt sacremoses googletrans==3.1.0a0 pandas sacrebleu matplotlib


In [ ]:
# Chargement des bibliothèques

# Fonctions auxilliaires prédéfinies
import nmt_dataset
import nnet_models

# PyTorch, une bibliothèque d'apprentissage machine similaire à Tensorflow (que Keras utilise)
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Bibliothèques Python standard
import os
import sys
import time
import copy
from functools import partial

# Chargement et traitement des données et des matrices
import numpy as np
import pandas as pd

# Visualisation sous forme de graphiques
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Barre de chargement pour les opérations lentes
from tqdm.notebook import tqdm

# Outil aidant à la construction du tokenizer
from subword_nmt.apply_bpe import BPE

# Instance de Google Traduction, qui nous servira de référence
from googletrans import Translator
translator = Translator()

%matplotlib inline

# Préparation du corpus de traduction

Le modèle que nous allons développer dans ce TP est un modèle de traduction de l'anglais vers le français. Afin de l'élaborer, nous utiliserons les données du projet [Tatoeba](https://tatoeba.org/fr), qui sont distribuées sous licence libre Creative Commons et hebergées par [Anki](www.manythings.org/anki).  

Pour pouvoir entraîner un modèle de traduction, il nous faut disposer d'un corpus *parallèle*, c'est-à-dire constitué d'une collection de phrases sources (`source_lang`) et de traductions de référence (`target_lang`). Par parallèle, on entend que ces deux collections doivent être alignées phrase par phrase. Dans notre cas, elles seront stockées dans deux fichiers, de telle sorte que chaque ligne du fichier de référence soit la traduction de la même ligne dans le fichier source.

Vous avez vu en cours que les corpus textuels doivent en général être pré-traités, ce que nous avons ignoré pour des raisons de simplicité au cours du TP précédent. Une des étapes les plus importantes du pré-traitement est la tokenization, qui consiste à découper chaque document (phrase) en plus petit morceaux appelés *tokens*. L'outil effectuant cette découpe s'appelle un *tokenizer*, et il en existe divers types, qui découpent par exemple le texte initial en mots, en sous-mots ou en caractères individuels. Ici, nous utiliserons un tokenizer en sous-mots de type *Byte Pair Encoding* (BPE), en utilisant la bibliothèque [`subword_nmt`](https://github.com/rsennrich/subword-nmt). Cette bibliothèque a la particularité qu'elle peut s'utiliser même hors de Python, directement dans le terminal. C'est ce que nous allons faire.

Comme le reste du modèle, le tokenizer doit être entraîné afin de déterminer quelles découpes ont du sens, en fonction des sous-mots qui sont les plus fréquents. Ici, le modèle que nous entraînons est un modèle BPE français-anglais, capable de faire la tokenization de phrases anglaises et françaises.

### Apprentissage du tokenizer
La cellule effectue les opérations suivantes :
- Passage en mode shell (exécution des commandes dans le terminal)
- Téléchargement et extraction des données si elles n'existent pas
- Séparation des données en jeux d'entraînement, de validation et de test (fonction auxilliaire contenue dans `prepare.py`)
- Entraînement du tokenizer

In [ ]:
%%shell
mkdir -p data

if [ ! -f data/train.en-fr.fr ]; then
    pushd data
    wget -nc https://www.manythings.org/anki/fra-eng.zip
    unzip -o fra-eng.zip
    popd
    python3 prepare.py data/fra.txt
fi

if [ ! -f data/bpecodes.en-fr ]; then
    cat data/train.en-fr.{en,fr} | subword-nmt learn-bpe -o data/bpecodes.en-fr -s 8000 -v
fi

In [ ]:
# Définition de quelques variables utiles, et lecture du corpus parallèle
data_dir = 'data/'
source_lang = 'en'
target_lang = 'fr'
model_dir = 'models/{}-{}'.format(source_lang, target_lang)

data_df = pd.read_csv(data_dir + 'fra.txt',
                      encoding='UTF-8', sep='\t', header=None,
                      names=['eng', 'fra'], index_col=False)
data_df

#### Questions (tokenizer)

Après avoir observé le résultat de l'exécution des deux cellules précédentes, répondre aux questions suivantes :

1. Quelle est la taille de ce corpus, et que remarquez-vous à son sujet ?
2. Comment fonctionne le tokenizer de type BPE ? Comment les différents sous-mots sont-ils construits ?
3. Cherchez (Ctrl-F/Cmd-F) dans le résultat de l'exécution de la cellule précédent le mot "danser". À quelle fréquence apparait-îl dans le corpus, et de la fusion de quels tokens est-il issu ?
4. Combien de tokens sont générés au total par l'algorithme de BPE ?  
5. Les sous-mots sont calculés en même temps sur les mots français et anglais. À votre avis, pourquoi est-ce le cas (pourquoi ne pas le faire séparément) ?

**Réponses :**
1. Le corpus contient 232 736 paires de phrases, ou 465 472 phrases (la moitié en français et la moitié en anglais). Parmi les points notables, on peut voir qu'une même phrase anglaise a plusieurs traductions valides en français. On sait donc d'ores et déjà qu'il sera impossible d'atteindre un score de traduction maximal (puisque le modèle prédira toujours la même traduction pour chaque phrase anglaise)
2. Le tokenizer BPE commence par créer un token pour chaque caractère possible ("a", "b", ..., "z", "A", ..., "Z", ".", "!", ...). Ensuite, on cherche la paire de tokens avec la fréquence la plus haute dans le corpus (par exemple "le", constitué de "l" et de "e"). On transforme cette paire en un nouveau token, et on répète cette étape jusqu'à avoir le nombre total de tokens souhaité.
3. La ligne `pair 3555: dan ser</w> -> danser</w> (frequency 172)` indique que le token "danser" est constitué de la fusion des tokens "dan" et "ser", eux-mêmes issues de la fusion de sous-tokens. Le mot "danser" apparaît 172 fois dans le corpus.
4. Ici, on a demandé à l'algorithme de générer 8000 tokens.
5. Le français et l'anglais sont issus de la même famille de langues (langues indo-européennes) et ont eu beaucoup d'influence mutuelle au cours de l'histoire. Il y a donc des similarités lexicales entre ces deux langues. Avoir les mêmes sous-mots pour les deux langues permet donc d'économiser de la mémoire, et peuvent aider le modèle à apprendre des caractéristiques communes à certains sous-mots entre les deux langues; par exemple, le suffixe -tion remplit globalement la même fonction en français et en anglais.

## Chargement et prétraitement des données

Le chargement et prétraitement des données consiste ici en 5 étapes :
1. Charger le tokenizer BPE en mémoire,
2. Charger les jeux d'entraînement, de validation et de test du corpus parallèle français-anglais. La fonction `load_data` va charger le corpus, puis le tokenizer en utilisant le fonction `preprocess`.
3. Créer ou charger le dictionnaire associant les tokens à leur identifiants uniques (IDs) (fonction `nmt_dataset.load_or_create_dictionary`)
4. Numériser les données en transformant chaque séquence source et cible en une séquences d'identifiants, et en les triant par taille (fonction `nmt_dataset.binarize`)
5. Créer les lots d'entraînement (batches) (classe `nmt_dataset.BatchIterator`) en groupant ensemble les séquences d'une taille similaire. Les séquences sont ensuite complétées de zéros (*padding*) afin qu'elles aient toutes la même longueur. Toutes les séquences sont contenues dans des matrices numpy, qui seront utilisées pour entrainer les modèles.

In [ ]:
# Reproducibilité des résultats
def reset_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)

#### 1. Chargement du tokenizer (bilingue français-anglais)

In [ ]:
bpe_path = os.path.join(data_dir, 'bpecodes.en-fr')

with open(bpe_path) as bpe_codes:
    bpe_model = BPE(bpe_codes)

# Fonction qui tokenize un document
def preprocess(line, is_source=True, source_lang=None, target_lang=None):
    return bpe_model.segment(line.lower())

# Fonction qui détokenize un document
def postprocess(line):
    return line.replace('@@ ', '')

def load_data(source_lang, target_lang, split='train', max_size=None):
    path = os.path.join(data_dir, '{}.{}-{}'.format(split, *sorted([source_lang, target_lang])))
    return nmt_dataset.load_dataset(path, source_lang, target_lang, preprocess=preprocess, max_size=max_size)

#### 2. Chargement et prétraitement du corpus parallèle
Comme évoqué plus haut, les données de notre corpus sont parallèles.
Le code ci-dessous charge les données dans les partitions `train`, `dev` et `test` et affiche un exemple de données source et cible avec le resultat de la tokenization.  

In [ ]:
train_data = load_data(source_lang, target_lang, 'train', max_size=None)
valid_data = load_data(source_lang, target_lang, 'valid')
test_data = load_data(source_lang, target_lang, 'test')
train_data.head()

##### Question (texte tokenisé)
1. À votre avis, à quoi sert le symbole `@@` dans les textes tokenizés ?

**Réponse :** Ce symbole représente la limite entre plusieurs tokens contenus dans un même mot. Pour reconstituer le mot, il faut donc coller tous les tokens puis supprimer ce symbole.

#### 3. Création/Chargement du dictionnaire

La prochaine étape est de determiner le vocabulaire utilisé dans les données. Ce vocabulaire est stocké dans deux listes (une pour chaque langue) appelées dictionnaires. Les dictionnaires sont triés par ordre de fréquence des mots.

In [ ]:
source_dict_path = os.path.join(model_dir, 'dict.{}.txt'.format(source_lang))
target_dict_path = os.path.join(model_dir, 'dict.{}.txt'.format(target_lang))

source_dict = nmt_dataset.load_or_create_dictionary(
    source_dict_path,
    train_data['source_tokenized'],
    minimum_count=10,
    reset=False
)

target_dict = nmt_dataset.load_or_create_dictionary(
    target_dict_path,
    train_data['target_tokenized'],
    minimum_count=10,
    reset=False
)
print(source_dict.words[:100])

print(target_dict.words[:100])
print('Source vocab size:', len(source_dict))
print('Target vocab size:', len(target_dict))

##### Question (dictionnaire)
1. Pourquoi un système de traduction automatique a-t-il besoin d'un dictionnaire ?
2. Pourquoi utilise-t-on un vocabulaire acquis sur les données d'apprentissage et non pas un vocabulaire standard, qui serait plus universel ?
3. À votre avis, que représente les quatre premiers éléments du vocabulaire ?
4. Quelle est la taille du vocabulaire pour chaque langue ?
5. En considerant le nombre de tokens connus par le tokenizer (section précédente) et la taille du vocabulaire, peut-on affirmer que les deux vocabulaires on des sous-mots en commun ?

**Réponses :**
1. Le dictionnaire permet tout simplement de convertir des mots (que l'on ne sait pas traiter de manière numérique) en indentifiants numériques (qui peuvent être traités). Il ne faut surtout pas répondre que le dictionnaire permet au système de connaître la traduction individuelle des mots, car ce n'est pas du tout le cas !
2. La raison principale est que d'apprendre un vocabulaire acquis sur les données d'apprentissage permet au modèle d'être adapté à ces données. En effet, différents mots ou expressions peuvent avoir des significations différentes selon le corpus. De plus, certains mots peuvent potentiellement n'apparaître que dans les données d'entraînement et pas dans le corpus universel.
3. `<sos>` (*start of sequence*) et `<eos>` (*end-of-sequence*) indiquent respectivement le début et la fin du document à traduire. `<unk>` (*unknown*) indique un mot ou token inconnu du dictionnaire (non présent dans les données d'entraînement). `<pad>` (*padding*) est un token de "bourrage", qui permet de remplir les séquences afin qu'elles aient toutes la même longueur.
4. On a 4 193 tokens pour l'anglais et 5 218 pour le français
5. 4 193 + 5 218 > 8 000. On a donc forcément des tokens en communs (plus exactement 1 411 tokens, soit 4 193 + 5 218 - 8 000) entre les deux langues.

#### 4. Utilisation du dictionnaire pour associer les tokens à leurs indices.


In [ ]:
nmt_dataset.binarize(train_data, source_dict, target_dict, sort=True)
nmt_dataset.binarize(valid_data, source_dict, target_dict, sort=False)
nmt_dataset.binarize(test_data, source_dict, target_dict, sort=False)
train_data.head()


#### Statistiques du corpus d'entrainement

In [ ]:
print('train_size={}, valid_size={}, test_size={}, min_len={}, max_len={}'.format(
    len(train_data),
    len(valid_data),
    len(test_data),
    train_data['source_len'].min(),
    train_data['source_len'].max(),
))

print('Distribution des longueurs de documents source (jeu d\'entraînement) :')
train_data['source_len'].describe()

##### Question (statistiques du corpus)
1. Combien de paires de phrases contient chacun des trois jeux de données (entraînement, validation et test) ?
2. Dans le jeu d'entraînement, quelle est la longueur en tokens du document source le plus court et de celui le plus long ? Quelle est la taille mediane en tokens des documents source ?

**Réponses :**
1. On a 228 736 paires d'entraînement, 2 000 paires de validation et 2 000 paires de test, soit un total de 232 726 paires comme vu précédemment
2. Dans le jeu d'entraînement, le document le plus court contient 3 tokens (dont `<sos>` et `<eos>`, donc un seul vrai token), le plus long en contient 85, et la longueur médiane est de 9 tokens.

#### 5. Construction des lots (batches)
Nous regroupons ici les différents documents en lots (*batches*), ce qui permettra d'entraîner notre modèle sur plusieurs documents en parallèle et donc plus rapidement.
Les lots sont automatiquement mélangés (shuffled) avant chaque itération d'entraînement.

In [ ]:
max_len = 30       # Maximum 30 tokens par document (les documents plus long seront tronqués)
batch_size = 512   # Maximum 512 tokens par lot

reset_seed()

train_iterator = nmt_dataset.BatchIterator(train_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=True)
valid_iterator = nmt_dataset.BatchIterator(valid_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=False)
test_iterator = nmt_dataset.BatchIterator(test_data, source_lang, target_lang, batch_size=batch_size, max_len=max_len, shuffle=False)

# Affiche le premier lot du jeu d'entraînement
print(next(iter(train_iterator)))

Le modèle séquence vers séquence (seq2seq)
=================

Une approche naïve pour traduire les phrases consisterait à traduire chaque mot séparément, puis à recoller les traductions des mots individuels bout à bout. Cependant, cela ne marche pas. Considérons la phrase "I am not the black cat" → "Je ne suis pas le chat noir". La plupart des mots de la phrase d'entrée ont une traduction directe dans la phrase de sortie, mais sont dans un ordre légèrement différents, par exemple "chat noir" et "black cat". De plus, du fait de la construction "ne/pas", il y a un mot de moins dans la phrase d'entrée.

Pour remédier à ce problème, un modèle seq2seq se compose généralement de deux parties, appelées *encodeur* et *décodeur*. L'encodeur lit une séquence d'entrée et produit un vecteur unique (encodage), qui correspond à une abstraction de la phrase d'entrée. Le décodeur lit ce vecteur pour produire une séquence de sortie, qui peut donc être de longueur et de forme différente de celle d'entrée.
Le modèle seq2seq est indépendant de la longueur et de l'ordre des séquences, ce qui le rend idéal pour la traduction entre deux langues.

![seq2seq](https://pytorch.org/tutorials/_images/seq2seq.png)

*Source de l'image: pytorch.org*

L'encodeur
-----------

L'encodeur est la partie qui prend une phrase et nous en donne une représentation (vecteur). Dans la version la plus simple, l'encodeur d'un réseau seq2seq peut être un RNN, qui produit une valeur pour chaque mot de la phrase d'entrée. Plus précisément, pour chaque mot d'entrée, l'encodeur produit un vecteur et un état caché, et utilise l'état caché pour prédire le mot d'entrée suivant.

Le décodeur
-----------

Le décodeur est un autre réseau de neurones, qui prend le(s) vecteur(s) de sortie de l'encodeur et produit une séquence de mots (traduction) à partir de celle-ci.

Dans le décodeur seq2seq le plus simple, nous utilisons uniquement la dernière sortie de l'encodeur. Cette dernière sortie est parfois appelée le *vecteur de contexte*, car elle encode le contexte de la séquence entière. Ce vecteur de contexte peut être utilisé comme état caché initial pour un décodeur, qui peut être par exemple de type RNN.

À chaque étape du décodage, le décodeur reçoit un token d'entrée et un état caché. Le token d'entrée initial est le token de début de chaîne et le premier état caché est le vecteur de contexte (le dernier état caché de l'encodeur).

Le décodeur
--------------------

Le décodeur est un autre réseau RNN qui prend le ou les vecteurs de sortie de l'encodeur et produit une séquence de mots pour générer la traduction.

Dans le décodeur seq2seq le plus simple, nous utilisons uniquement la dernière sortie de l'encodeur. Cette dernière sortie est parfois appelée le vecteur de contexte car elle code le contexte de la séquence entière. Ce vecteur de contexte peut être utilisé comme état caché initial pour un décodeur RNN.

À chaque étape du décodage, le décodeur reçoit un token d'entrée et un état caché. Le token d'entrée initial est le token de début de chaîne et le premier état caché est le vecteur de contexte (le dernier état caché de l'encodeur).

![Illustration encodeur-decodeur (si elle ne se charge pas, ouvrez l'image seq2seq.png)](seq2seq.png)

## Encodeur et décodeur RNN

In [ ]:
rnn_encoder = nnet_models.RNN_Encoder(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.2
)

print(rnn_encoder)

In [ ]:
rnn_decoder = nnet_models.RNN_Decoder(
    output_size=len(target_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.2
)

print(rnn_decoder)

In [ ]:
rnn_model = nnet_models.EncoderDecoder(
    rnn_encoder,
    rnn_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict
)

#### Questions (paramètres du modèle)

À quoi correspondent les paramètres suivants ?
1. `input_size=len(source_dict)`
2. `output_size=len(target_dict)`
3. `hidden_size=512`
4. `num_layers=1`
5. `dropout=0.2`
6. `lr=0.001`

**Réponses :**
1. L'encodeur prend une entrée de type séquence de tokens. Le nombre d'identifiants possibles pour chaque token correspond à la taille du dictionnaire source (taille d'entrée).
2. Idem pour la sortie du décodeur, avec le dictionnaire cible cette fois.
3. La représentation d'un document (en sortie de l'encodeur/entrée du décodeur) sera un vecteur de taille 512.
4. On a une seule couche cachée dans l'encodeur et le décodeur.
5. 20% des neurones seront aléatoirement ignorés pendant chaque itération de l'entraînement afin de régulariser le réseau de neurones.
6. Le taux d'apprentissage est de 0.001.

# Apprentissage du modèle de traduction

## Programmation de l'apprentissage

Comme dans les TP précédents, l'apprentissage s'effectue durant un certain nombre d'itérations. Pour verifier si l'apprentissage évolue positivement, nous utilisons deux mesures pour évaluer le modèle à chaque itération :
- La fonction de coût du modèle
- La mesure BLEU, qui est une métrique couramment utilisée en traduction. Cette métrique compare la sortie du modèle à une traduction de référence, et lui attribue un score.

Le code ci-dessous sert à effectuer l'apprentissage du modèle :

In [ ]:
def save_model(model, checkpoint_path):
    # Fonction servant à enregistrer le réseau de neurones
    dirname = os.path.dirname(checkpoint_path)
    if dirname:
        os.makedirs(dirname, exist_ok=True)
    torch.save(model, checkpoint_path)

def train_model(
        train_iterator,
        valid_iterators,
        model,
        checkpoint_path,
        epochs=10,
        validation_frequency=1
    ):

    reset_seed()  # Reproducibilité
    epochsX = list(range(1, epochs+1))
    epoch_lossY = []
    epoch_loss_validation = []

    best_bleu = -1
    for epoch in range(1, epochs + 1):

        start = time.time()
        running_loss = 0
        running_loss_dev = 0

        print('Itération : [{}/{}]'.format(epoch, epochs))

        # Enumère tous les lots d'entraînement
        for i, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
            t = time.time()
            running_loss += model.train_step(batch)

        # Calcule la fonction de coût moyenne sur le jeu d'entraînement pour cette itération
        epoch_loss = running_loss / len(train_iterator)
        epoch_lossY.append(epoch_loss)

        print("Coût (train)={:.3f}, durée={:.2f}".format(epoch_loss, time.time() - start))

        # Enumère tous les lots de validation
        for i, batch in tqdm(enumerate(valid_iterators[0]), total=len(valid_iterators[0])):
            t_dev = time.time()
            running_loss_dev += model.dev_step(batch)

        # Calcule la fonction de coût moyenne sur le jeu de validation pour cette itération
        epoch_loss_dev = running_loss_dev / len(valid_iterators[0])
        epoch_loss_validation.append(epoch_loss_dev)

        print("Coût (valid)={:.3f}, durée={:.2f}".format(epoch_loss_dev, time.time() - start))

        sys.stdout.flush()

        # Évalue et enregistre le modèle
        if epoch % validation_frequency == 0:
            bleu_scores = []

            # Calcule le score BLEU sur tous les lots de validation
            for valid_iterator in valid_iterators:
                src, tgt = valid_iterator.source_lang, valid_iterator.target_lang
                translation_output = model.translate(valid_iterator, postprocess)
                bleu_score = translation_output.score
                output = translation_output.output

                with open(os.path.join(model_dir, 'valid.{}-{}.{}.out'.format(src, tgt, epoch)), 'w') as f:
                    f.writelines(line + '\n' for line in output)

                print('{}-{}: BLEU={}'.format(src, tgt, bleu_score))
                sys.stdout.flush()
                bleu_scores.append(bleu_score)

            # Calcule le score BLEU moyen obtenu
            bleu_score = round(sum(bleu_scores) / len(bleu_scores), 2)
            if len(bleu_scores) > 1:
                print('BLEU={}'.format(bleu_score))

            # Met à jour le taux d'apprentissage du réseau de neurones en fonction du score BLEU.
            # Ici, le taux d'apprentissage est divisé par 10 si le score BLEU n'augmente pas.
            model.scheduler_step(bleu_score)

            # Si le réseau de neurones obtient un score BLEU plus élevé qu'à toutes les itérations précédentes, on l'enregistre
            if bleu_score > best_bleu:
                best_bleu = bleu_score
                save_model(model, checkpoint_path)

        print('=' * 50)

    print("Apprentissage terminé. Meilleure score BLEU : {}".format(best_bleu))

    plt.plot(epochsX, epoch_lossY, '--x', label='Training Loss')
    plt.plot(epochsX, epoch_loss_validation, '--x', label='Validation Loss')
    plt.ylim(0, np.max(epoch_lossY) + 1)
    plt.xticks(range(1,epochs+1))
    plt.xlabel('Epochs')
    plt.ylabel('Epoch Loss')
    plt.legend()

Note : Il faut savoir que les termes _dev_ et _validation_ désignent le même jeu de données.

#### Question (score BLEU)
1. Entre quelle valeur et quelle valeur peut être compris un score BLEU ? Vaut-il mieux que la valeur obtenue soit élevée ou faible ? Pourquoi ?

**Réponse :** Un score BLEU peut aller de 0 à 1, ou de 0 à 100 en mode pourcentage. Une valeur élevée correspond à une meilleure performance, car cela signifie que la phrase évaluée est plus proche de la phrase de référence. Cependant, il s'agit d'une comparaison purement syntaxique; un modèle qui génère des traductions parfaites mais formulées différemment de celles de référence obtiendrait donc un score plus bas que ce à quoi on peut s'attendre.

## Execution de l'apprentissage du modèle de traduction

**Remarque :** l'apprentissage peut durer longtemps (plusieurs minutes). Pour éviter cela, vous pouvez régler `epoch` (le nombre d'itérations) sur une petite valeur, par exemple 2, et relancer cette cellule plusieurs fois pour continuer à entraîner votre modèle (`train_model` ne réinitialise pas le modèle). Si l'apprentissage dure plus de 10 minutes, vérifiez que vous avez bien sélectionne un GPU comme demandé au tout début du TP.

In [ ]:
checkpoint_path = os.path.join(model_dir, 'rnn.pt')

train_model(train_iterator, [valid_iterator], rnn_model,
            epochs=10,
            checkpoint_path=checkpoint_path)

#### Question (entraînement)
Observez la figure présentant la fonction de coût (*loss*) à chaque itération (au moins 5 itérations au total), puis répondez aux questions suivantes :
1. Comment le coût évolue-t-il avec le nombre d'époques ?
2. Est-ce que le modèle pourrait poursuivre son entrainement, ou au contraire, voit-on apparaitre un sur-apprentissage ?

**Réponses :**
1. Les deux coûts ont tendance à diminuer au fur et à mesure que les époques sont effectuées, mais le coût de validation stagne rapidement.
2. Le coût de validation commence à stagner après 6 époques, il est donc possible que l'on voie le début d'un sur-apprentissage.

## Evaluation du modèle sur le corpus de test

In [ ]:
print('BLEU:', rnn_model.translate(test_iterator, postprocess).score)

## Test interactif du modèle

Vous pouvez tester votre modèle de manière interactive soumettant des phrases à traduire de votre choix. Il n'est pas nécessaire de lire ou comprendre le code ci-dessous, il suffit de modifier les phrases de test dans le bloc de code situé après les définitions de fonctions.

In [ ]:
def get_binned_bleu_scores(model, valid_iterator):
    lengths = np.arange(4, 20, 3)
    bleu_scores = np.zeros(len(lengths))

    for i in tqdm(range(1, len(lengths)), total=len(lengths) - 1):
        min_len = lengths[i - 1]
        max_len = lengths[i]

        tmp_data = valid_data[(valid_iterator.data['source_len'] > min_len) & (valid_iterator.data['source_len'] <= max_len)]
        tmp_iterator = nmt_dataset.BatchIterator(tmp_data, source_lang, target_lang, batch_size, max_len=max_len)

        bleu_scores[i] = model.translate(tmp_iterator, postprocess).score

    lengths = lengths[1:]
    bleu_scores = bleu_scores[1:]

    plt.plot(lengths, bleu_scores, 'x-')
    plt.ylim(0, np.max(bleu_scores) + 1)
    plt.xlabel('Source length')
    plt.ylabel('BLEU score')

    return lengths, bleu_scores


def show_attention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions, cmap='bone', aspect='auto')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + input_sentence.split(' ') + [nmt_dataset.EOS_TOKEN], rotation=90)
    ax.set_yticklabels([''] + output_words.split(' ') + [nmt_dataset.EOS_TOKEN])

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def encode_as_batch(sentence, dictionary, source_lang, target_lang):
    sentence = sentence + ' ' + nmt_dataset.EOS_TOKEN
    tensor = dictionary.txt2vec(sentence).unsqueeze(0)

    return {
        'source': tensor,
        'source_len': torch.from_numpy(np.array([tensor.shape[-1]])),
        'source_lang': source_lang,
        'target_lang': target_lang
    }


def get_translation(model, sentence, dictionary, source_lang, target_lang, return_output=False):
    print('Source:', sentence)
    sentence_tok = preprocess(sentence, is_source=True, source_lang=source_lang, target_lang=target_lang)
    print('Tokenized source:', sentence_tok)
    batch = encode_as_batch(sentence_tok, dictionary, source_lang, target_lang)
    prediction, attn_matrix, enc_self_attn = model.eval_step(batch)
    prediction = prediction[0]
    prediction_detok = postprocess(prediction)
    print('Prediction:', prediction)
    print('Detokenized prediction:', prediction_detok)

    print('Google Translate ({}->{}): {}'.format(
        source_lang,
        target_lang,
        translator.translate(sentence, src=source_lang, dest=target_lang).text
    ))
    print('Google Translate on prediction ({}->{}): {}'.format(
        target_lang,
        source_lang,
        translator.translate(prediction_detok, src=target_lang, dest=source_lang).text
    ))

    results = {
        'source': sentence,
        'source_tokens': sentence_tok.split(' ') + ['<eos>'],
        'prediction_detok': prediction_detok,
        'prediction_tokens': prediction.split(' '),
    }

    if attn_matrix is not None:
        attn_matrix = attn_matrix[0].detach().cpu().numpy()
        results['attention_matrix'] = attn_matrix
        show_attention(sentence_tok, prediction, attn_matrix)

    if enc_self_attn is not None:
        results['encoder_self_attention_list'] = enc_self_attn

    if return_output:
        return results

In [ ]:
get_translation(rnn_model, 'hello how are you ?', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_model, 'are hello ? how you', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_model, 'she \'s five years older than me .', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_model, 'i know that the last thing you want to do is help me .', source_dict, source_lang, target_lang)

## Tracé du score BLEU en fonction de la longueur de la séquence source

Comme vu en cours, les performances des modèles RNNs chutent lorsque la longueur de l'entrée augmente.
Ceci est dû à trois facteurs principaux :
- Le décodeur RNN se base uniquement sur le dernier état caché de l'encodeur. Cela signifie que nous devons encoder la phrase complète dans un vecteur unique de taille fixe. Plus la phrase est longue, plus cela est difficile.
- Les RNN encodeurs-décodeurs sont difficiles à entraîner (car le signal doit être rétro-propagé à travers toute la séquence d'états).
- L'ensemble d'apprentissage que nous avons utilisé est principalement composé de phrases très courtes (95 % des phrases sources comportent 15 éléments ou moins).

Vous pouvez visualiser ce phénomène sur le graphique suivant :

In [ ]:
rnn_lengths, rnn_bleu_scores = get_binned_bleu_scores(rnn_model, valid_iterator)

Le modèle séquence-vers-séquence (seq2seq) avec attention
=================


Le vecteur passé au décodeur doit représenter l'ensemble des informations de l'entrée, ce qui est trop peu, notamment pour les phrases longues. Le mécanisme d'attention est un moyen de recalculer un lien avec les éléments importants de l'entrée à chaque pas de décodage. Ainsi, dans la figure ci-dessous, le terme "Je" dépend plus fortement de termes "I" et "am" que de "a" et "student". Ceci permet au modèle d'être plus flexible et de prendre en compte des dépendances qui peuvent être assez longues.



![seq2seq_avec_attention](https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg)



*Source de l'image: www.tensorflow.org*




Les architectures avec mécanismes d'attention connaissent beaucoup de succès dans de nombreuses applications comme la traduction, la reconnaissance de la parole, la description d'image, etc. ([cet article de blog constitue une excellente introduction en la matière](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)).

Ici, nous n'avons pas besoin de modifier l'architecture de l'encodeur existant. En revanche, le décodeur fonctionne différemment : en ajoutant un mécanisme d'attention, à chaque étape du décodage, le décodeur reçoit un token d'entrée, l'état caché précédent et un vecteur qui est calculé directement à partir de la séquence d'entrée. Ainsi, à chaque étape de décodage, le décodeur peut donner plus ou moins d'**attention** à des tokens de la source, ce qui permet de prendre en compte des dépendances plus longues que dans la version sans attention.

## Encodeur et décodeur RNN avec attention

In [ ]:
rnn_attn_encoder = nnet_models.RNN_Encoder(
    input_size=len(source_dict),
    hidden_size=512,
    num_layers=1,
    dropout=0.2
)

print(rnn_attn_encoder)

In [ ]:
rnn_attn_decoder = nnet_models.AttentionDecoder(
    output_size=len(target_dict),
    hidden_size=512,
    dropout=0.2
)

print(rnn_attn_decoder)

In [ ]:
rnn_attn_model = nnet_models.EncoderDecoder(
    rnn_attn_encoder,
    rnn_attn_decoder,
    lr=0.001,
    use_cuda=True,
    target_dict=target_dict
)

## Apprentissage du modèle

**Remarque :** l'entrainement est encore plus long que précédemment (3 minutes par itération). À nouveau, il est recommandé de procéder en n'entraînant que pendant une ou deux itérations à la fois.

In [ ]:
checkpoint_path = os.path.join(model_dir, 'rnn_attn.pt')

train_model(train_iterator, [valid_iterator], rnn_attn_model,
            epochs=1,
            checkpoint_path=checkpoint_path)

## Évaluation du modèle sur le corpus de test

In [ ]:
print('BLEU:', rnn_attn_model.translate(test_iterator, postprocess).score)

#### Question (encodeur avec attention)

*Il est possible de répondre à cette question après une seule itération d'apprentissage.*

1. Après avoir ajouté le mécanisme d'attention, observez-vous un gain de performance ? À votre avis, pourquoi ?

**Réponse :** Dépend de votre machine. En général, on peut observer un gain de performance assez faible. On peut justifier que les performances sont meilleures (car l'attention est plus efficace) ou pas (car le modèle est trop petit, on n'utilise pas assez de données d'entraînement, le score BLEU n'est pas adapté, l'attention a besoin de plus de temps pour être apprise...)

## Test interactif du modèle et visualisation des matrices d'attention

Vous pouvez tester votre modèle de manière interactive grâce au code ci-dessous, en soumettant des phrases à traduire de votre choix.

Pour chacune des traductions ci-dessous, vérifiez si le mécanisme d'attention permet bien de focaliser la prédiction du prochain mot dans la langue cible sur les mots importants de la séquence d'entrée dans la langue source.

In [ ]:
get_translation(rnn_attn_model, 'hello how are you ?', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_attn_model, 'are hello ? how you', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_attn_model, 'she \'s five years older than me .', source_dict, source_lang, target_lang)

In [ ]:
get_translation(rnn_attn_model, 'i know that the last thing you want to do is help me .', source_dict, source_lang, target_lang)

# Bonus : Optimisation des paramètres

Pour l'instant, le modèle obtient des scores moyens. Charge à vous de faire évoluer la configuration afin d'obtenir un modèle plus performant. Plusieurs paramètres peuvent être modifiés afin d'obtenir de meilleures traductions. Dans le premier TP, nous avons fait des expérimentations en changeant les paramètres `epoch` et `lr`. Dans ce TP, essayez de faire évoluer les paramètres décrits ci-après et observez la performance du modèle.

Afin de bien comprendre l'importance de chaque paramètre dans la performance globale du système, veillez dans un premier temps à changer les paramètres un à un (et non tous en même temps) et notez les résultats. De la sorte, vous pourrez quantifier précisément le rôle de chaque paramètre.

Vous pouvez également faire varier les paramètres suivants :

- **Taille des lots** : Faites varier `batch_size` (ex. 512 -> 256 -> 128, n'allez pas au-delà de 512).
- **Nombre de couches** : Augmentez `num_layers` (n'allez pas au-delà de 3 couches)
- **Nombre de neurones** : Faites varier `hidden_size` (ex. 512 -> 256 -> 128, n'allez pas au-delà de 512)
- **Dropout** : Modifiez la valeur de `dropout` (ex. 0.0 -> 0.1 -> 0.2)
- **Taille de la partition dev** : Initialement `train_size=186206`, `valid_size=2000`, `test_size=2000`, `min_len=3`, `max_len=69`. Vous pouvez modifier le nombre des données de validation dans le fichier `prepare.py` et recharger le fichier.

En tenant compte des paramètres optimaux de ces TPs, essayez de choisir la combinaison de paramètres la plus efficace possible pour atteindre le meilleur score BLEU. Vous pouvez changer les paramètres que vous venez de tester ainsi que les autres paramètres disponibles pour l'architecture du modèle, son entraînement et le décodage.


### Question (optimisation)
1. À la fin de ce TP, quel est le meilleur score BLEU que vous pouvez obtenir ?

[ **répondre ici** ]